In [1]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pickle
import string  
import annoy  # библиотека построения индекса приближенного поиска ближайших соседей

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from gensim.models import Word2Vec
from tqdm import tqdm_notebook

In [5]:
# Проверяем GPU count, name
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [6]:
# Подколючаем доступ к Google-Диску
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
checks = pd.read_csv("checks.csv")
checks.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"


In [14]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    """Функция предобработки текстов"""
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

Значения столбцов в датасете:

1) sale_date_date - дата

2) contact_id - id кассы

3) shop_id - id магазина

4) product_id - id товара

5) name - название товара

6) product_sub_category_id - подкатебгория товара

7) product_category_id - категория товара

8) brand_id - бренд товара

9) quantity - кол-во товаров

In [12]:
# Смотрим перечень товаров
with open("Product_dict.pkl","rb") as f:
    product_dict = pickle.load(f)

In [4]:
# Уберем пропуски в данных по 3 столбцам
checks.dropna(axis = 0,subset = ['sale_date_date', 
                                 'contact_id', 
                                 'shop_id'], inplace = True)

# Введем уникальный идентификаторв чека:
checks['check_id'] = checks['sale_date_date'].apply(str) + '_' + \
                     checks['contact_id'].apply(str) + '_' + \
                     checks['shop_id'].apply(str)

# Отсортируем по дате для определения валидационной части
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', ascending = True, inplace = True) 

# Поменяем тип product_id в int
checks['product_id'] = checks['product_id'].apply(lambda x: int(x))

# Разбиваем train-test 
from sklearn.model_selection import train_test_split
train, test = train_test_split(checks, test_size = 0.3, shuffle = False)

## Реализация контентных рекомедаций по товарам из product_dict (поиск ближайших похожих товаров)

In [15]:
# Обработаем словарь товаров
product_df = pd.DataFrame.from_dict(product_dict, orient = 'index', columns = ['name'])
product_df.reset_index(inplace = True)
product_df.rename(columns={'index': 'product_id'}, inplace = True)
product_df['text'] = product_df['name'].apply(lambda x: preprocess_txt(str(x)))

# Обучим модель FastText на name, длина которых > 3, меньше - бессмыслица
sentences = [i for i in product_df['text'] if len(i) > 3]
modelFT = FastText(sentences = sentences,
                   vector_size = 20, 
                   min_count = 1, 
                   window = 5)

In [61]:
# # Обучим модель Word2Vec на name, длина которых > 3, меньше - бессмыслица
modelW2V_products = Word2Vec(sentences = sentences,
                    vector_size = 20, 
                    min_count = 1, 
                    window = 5)

In [16]:
# Для FastText
# Для того, чтобы быстро находить товары положим эмбединги их name в ANN индекс
# Создадим объект индекса
ft_index_product = annoy.AnnoyIndex(20 ,'angular')

index_map_product = {}
counter = 0

for i in tqdm_notebook(range(len(product_df))):
    n_ft = 0
    index_map_product[counter] = (product_df.loc[i, "text"],
                                  product_df.loc[i, "product_id"],
                                  product_df.loc[i, "name"])
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in product_df.loc[i, "text"]:
        if word in modelFT.wv.key_to_index:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_product.add_item(counter, vector_ft)
    counter += 1


ft_index_product.build(10)

<ipython-input-16-a66bf4867325>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(product_df))):


  0%|          | 0/30418 [00:00<?, ?it/s]

True

In [62]:
# Для Word2Vec
# Для того, чтобы быстро находить товары положим эмбединги их name в ANN индекс
# Создадим объект индекса
index_product_w2v = annoy.AnnoyIndex(20 ,'angular')

index_map_product_w2v = {}
counter_w2v = 0

for i in tqdm_notebook(range(len(product_df))):
    n_w2v = 0
    index_map_product_w2v[counter_w2v] = (product_df.loc[i, "text"],
                                          product_df.loc[i, "product_id"],
                                          product_df.loc[i, "name"])
    vector_w2v = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in product_df.loc[i, "text"]:
        if word in modelW2V_products.wv.key_to_index:
            vector_w2v += modelW2V_products.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    index_product_w2v.add_item(counter_w2v, vector_w2v)
    counter_w2v += 1


index_product_w2v.build(10)

<ipython-input-62-56f524f8ab47>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(product_df))):


  0%|          | 0/30418 [00:00<?, ?it/s]

True

In [17]:
def recommend_single(product_id):
    """Функция рекомендации товара"""
    # Получим name айтема по идентификатору
    text = product_df[product_df['product_id']==product_id]['text'].values[0]
    
    vector_ft = np.zeros(20)
    n_ft = 0
    # Каждое слово обернем в эмбеддинг
    for word in text:
        if word in modelFT.wv.key_to_index:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    # Найдем ближайшие по name
    rec_items = ft_index_product.get_nns_by_vector(vector_ft, 10)
    return [index_map_product[i][2] for i in rec_items]

In [18]:
recommend_single("221495")

['(7.2.A6.201) Гидроксипрогестерон (17-OH-прогестерон) - Тесты репродукции -1',
 '(7.2.A7.201) Андростендион - Тесты репродукции -1',
 '(7.2.A4.201) Эстрадиол ( Е2) - Тесты репродукции -1',
 '(7.2.A5.201) Прогестерон - Тесты репродукции -1',
 '(0000122403 ) Ранункулус микс -1',
 '(0000122813 ) Ваксфлауэр микс -1',
 '(00000013266) Амарилис микс -1',
 '(47453) Сателлит ПКГ-02 Глюкометр -1',
 '(0000122441 ) Орхидея микс -1',
 '(7.2.A8.201) Дегидроэпиандростерон сульфат ( ДГЭА-сульфат ) - Тесты репродукции -1']

## Реализация рекомедаций по чековым последовательностям из train (ниже загружаем уже сформированные группы из чековых данных)

In [15]:
# # Уберем лишние столбцы
# train.drop(columns = ['sale_date_date', 
#                        'contact_id', 
#                        'shop_id', 
#                        'product_sub_category_id', 
#                        'product_category_id',
#                        'brand_id',
#                        'quantity'
#                        ], axis=1, inplace=True)
# # train['product_id'] = train['product_id'].apply(lambda x: str(x))

In [16]:
# # Сгруппируем значения product_id по check_id
# agg_func_text = {'product_id': set}
# grouped_train = train.groupby(['check_id']).agg(agg_func_text)


# Сгруппируем значения name по check_id
# agg_func_text = {'name': set}
# grouped_train = train.groupby(['check_id']).agg(agg_func_text)

# grouped_train['name'] = grouped_train['name'].apply(lambda x: preprocess_txt(x))
# grouped_train

In [3]:
grouped_train = pd.read_csv("grouped_train_final.csv")
grouped_train.head()

,Unnamed: 0,name
0,0,"['39616', 'пектусин', 'табдрассас№10', '109979..."
1,1,"['52707', 'зодак', 'таб', '10', 'мг', '№30', '..."
2,2,"['119158', 'минк', 'мыло', 'сливочный', 'печен..."
3,3,"['111657', 'кодальнев', 'таб5мг25мг8мг', '№30'..."
4,4,"['106041', 'белый', 'уголь', 'актив', 'таб700м..."


In [6]:
grouped_train['name'][0].strip("'[]").split("', '")

['39616',
 'пектусин',
 'табдрассас№10',
 '109979',
 'лортенз',
 'табппо5мг50мг',
 '№30']

In [7]:
grouped_train['name'] = grouped_train['name'].apply(lambda x: x.strip("'[]").split("', '"))

In [ ]:
# # Обучим модель Word2Vec на product_id, длина которых > 1, меньше - бессмыслица
# sentences_w2v = [i for i in grouped_train['product_id']]
# modelW2V = Word2Vec(sentences = sentences_w2v,
#                     size = 20, 
#                     min_count = 1, 
#                     window = 5)

In [10]:
# Обучим модель Word2Vec на name
sentences_w2v = [i for i in grouped_train['name']]
modelW2V = Word2Vec(sentences = sentences_w2v,
                    vector_size = 20, 
                    min_count = 1, 
                    window = 5)

In [ ]:
# def recommend_check_id(*args):
#     """Функция рекомендации товара по чековой последовательности"""
#     lst = []
#     for arg in args:
#       lst.append(str(int(arg)))
   
#     vector_w2v = np.zeros(20)
#     n_w2v = 0

#     # Каждое слово обернем в эмбеддинг
#     for word in lst:
#         if word in modelW2V.wv.key_to_index:
#             vector_w2v += modelW2V.wv[word]
#             n_w2v += 1
#     if n_w2v > 0:
#         vector_w2v = vector_w2v / n_w2v
    
#     # Найдем ближайшие по product_id
#     rec_items = ft_index_product.get_nns_by_vector(vector_w2v, 20)
#     return [index_map_product[i][1] for i in rec_items]

In [66]:
def recommend_check_name(*args):
    """Функция рекомендации товара по чековой последовательности"""
    lst = []
    for arg in args:
        for i in range(len(product_df[product_df['product_id']==str(arg)]['text'].values[0])):
            lst.append(product_df[product_df['product_id']==str(arg)]['text'].values[0][i])
   
    vector_w2v = np.zeros(20)
    n_w2v = 0

    # Каждое слово обернем в эмбеддинг
    for word in lst:
        if word in modelW2V.wv.key_to_index:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    
    # Найдем ближайшие по product_id
    rec_items = index_product_w2v.get_nns_by_vector(vector_w2v, 20)
    return [index_map_product_w2v[i] for i in rec_items]

In [ ]:
# Тестируем ответы на test
test.drop(columns = ['sale_date_date', 
                       'contact_id', 
                       'shop_id', 
                       'product_sub_category_id', 
                       'product_category_id',
                       'brand_id',
                       'quantity'
                       ], axis=1, inplace=True)
test['product_id'] = test['product_id'].apply(lambda x: str(x))

# Группируем product_id на test
agg_func_text = {'product_id': set}
grouped_test = test.groupby(['check_id']).agg(agg_func_text)
grouped_test['product_id'] = grouped_test['product_id'].apply(lambda x: list(x))
grouped_test.tail(20)

In [ ]:
# Пробуем работу рекомендательной системы по факту test

In [71]:
product_df[product_df['product_id']==str(109489)]['text'].values[0]

['72688', 'дротаверин', 'гидрохлорид', 'тб', '40мг', 'n50', '397']

In [70]:
product_df[product_df['product_id']==str(116291)]['text'].values[0]

['96971', 'анальгин', 'таб500мг', '№20', '397']

In [72]:
product_df[product_df['product_id']==str(120163)]['text'].values[0]

['68077',
 'пластырькомпресс',
 'экстрапласт',
 'температура',
 '5',
 'х',
 '12см',
 '№3',
 '670']

In [67]:
recommend_check_name(116291, 109489)
#120163

[(['179514', 'фенхель', 'масло', 'эфирный', 'фл10мл', '410'],
  '155758',
  '(179514) Фенхель масло эфирное фл.10мл 410'),
 (['116972', 'грейпфрут', 'эфирный', 'масло', 'фл10мл', '410'],
  '75322',
  '(116972) Грейпфрут эфирное масло фл.10мл 410'),
 (['179458', 'ель', 'масло', 'эфирный', '10мл', '410'],
  '155867',
  '(179458) Ель масло эфирное 10мл 410'),
 (['179461', 'жожоба', 'масло', 'эфирный', '10мл', '410'],
  '155943',
  '(179461) Жожоба масло эфирное 10мл 410'),
 (['196126', 'илангиланг', 'масло', 'эфирный', '10мл', '410'],
  '193745',
  '(196126) Иланг-иланг масло эфирное 10мл 410'),
 (['179561', 'нероли', 'масло', 'эфирный', '10мл', '410'],
  '155906',
  '(179561) Нероли масло эфирное 10мл 410'),
 (['196131', 'розмарин', 'масло', 'эфирный', '10мл', '410'],
  '213310',
  '(196131) Розмарин масло эфирное 10мл 410'),
 (['179313', 'жасмина', 'масло', 'эфирный', '10мл', '410'],
  '155614',
  '(179313) Жасмин масло эфирное 10мл 410'),
 (['200867', 'блюменберг', 'масло', 'эфирный', 

In [75]:
product_df[product_df['product_id']==str(120163)]['text'].values[0]

['68077',
 'пластырькомпресс',
 'экстрапласт',
 'температура',
 '5',
 'х',
 '12см',
 '№3',
 '670']

In [74]:
product_df[product_df['product_id']==str(109489)]['text'].values[0]

['72688', 'дротаверин', 'гидрохлорид', 'тб', '40мг', 'n50', '397']

In [73]:
product_df[product_df['product_id']==str(116291)]['text'].values[0]

['96971', 'анальгин', 'таб500мг', '№20', '397']

In [69]:
recommend_check_name(120163, 109489)
#116291

[(['209409', 'марвис', 'зубной', 'паста', 'мятажасмина', '85мл', '529'],
  '224397',
  '(209409) Марвис Зубная паста Мята-Жасмин 85мл 529'),
 (['195865', 'апагарда', 'мплюс', 'зубной', 'паста', 'отбеливающ125мл', '529'],
  '192226',
  '(195865) Апагард М-Плюс Зубная паста отбеливающ.125мл 529'),
 (['209407', 'марвис', 'зубной', 'паста', 'мятаимбирь', '85мл', '529'],
  '224278',
  '(209407) Марвис Зубная паста Мята-Имбирь 85мл 529'),
 (['209410', 'марвис', 'зубной', 'паста', 'мятакорица', '85мл', '529'],
  '223052',
  '(209410) Марвис Зубная паста Мята-Корица 85мл 529'),
 (['195866',
   'апагарда',
   'премио',
   'зубной',
   'паста',
   'отбеливающая100мл',
   '529'],
  '212952',
  '(195866) Апагард Премио Зубная паста отбеливающая100мл 529'),
 (['36460', 'эльгидиум', 'зубной', 'паста', '100г75мл', '529'],
  '65573',
  '(36460) Эльгидиум Зубная паста 100г/75мл 529'),
 (['60782',
   'зубной',
   'паста',
   'сплат',
   'special',
   'dreamмечта',
   'артm130',
   '529'],
  '61480',
  '

## Выводы:
Самое важное в данном проекте рекомендательной системы то, что нужно уделять много времени валидации результатов, поскольку изначально определена валидационная выборка, как более поздние данные о продажах. Для работы в боевом режиме этому аспекту неюходимо уделить колоссальное количество времени и ресурсов, поскольку признаков, влияющих на результат предсказания можно спроектировать гораздо больше, чем просто имя товара. Например, категории и подкатегории товара, что, несомненно, оказывает влияние на результаты.